# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hopspitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hopspitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-30-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-30-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-01 04:33:52,34.223334,-82.461707,113,0,0,113,"Abbeville, South Carolina, US",460.716761,0.000000
1,22001.0,Acadia,Louisiana,US,2020-07-01 04:33:52,30.295065,-92.414197,878,36,0,842,"Acadia, Louisiana, US",1415.101942,4.100228
2,51001.0,Accomack,Virginia,US,2020-07-01 04:33:52,37.767072,-75.632346,1042,14,0,1028,"Accomack, Virginia, US",3224.408962,1.343570
3,16001.0,Ada,Idaho,US,2020-07-01 04:33:52,43.452658,-116.241552,2169,23,0,2146,"Ada, Idaho, US",450.385912,1.060396
4,19001.0,Adair,Iowa,US,2020-07-01 04:33:52,41.330756,-94.471059,15,0,0,15,"Adair, Iowa, US",209.731544,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until day is over.**

In [6]:
jh_cases_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_cases_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,28833,29157,29481,29640,30175,30451,30616,30967,31238,31517
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1962,1995,2047,2114,2192,2269,2330,2402,2466,2535
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,11771,11920,12076,12248,12445,12685,12968,13273,13571,13907


In [7]:
jh_deaths_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_deaths_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,581,598,618,639,675,683,703,721,733,746
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,44,44,45,47,49,51,53,55,58,62
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,845,852,861,869,878,885,892,897,905,912


In [8]:
jh_recovered_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_recovered_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,8764,8841,9260,9869,10174,10306,10674,12604,13934,14131
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1134,1159,1195,1217,1250,1298,1346,1384,1438,1459
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,8422,8559,8674,8792,8920,9066,9202,9371,9674,9897


In [9]:
jh_cases_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_cases_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,222,222,225,226,231,247,247,247,253,257
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,30,30,30,30,30,30,30,30,30,30


In [10]:
jh_deaths_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_deaths_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [11]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
1738,1093.0,Marion,Alabama,US,2020-07-01 04:33:52,34.136974,-87.887042,200,12,0,188,"Marion, Alabama, US",673.196674,6.000000
337,1015.0,Calhoun,Alabama,US,2020-07-01 04:33:52,33.774837,-85.826304,268,5,0,263,"Calhoun, Alabama, US",235.905110,1.865672
2197,1107.0,Pickens,Alabama,US,2020-07-01 04:33:52,33.280949,-88.088181,204,6,0,198,"Pickens, Alabama, US",1023.582539,2.941176


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [12]:
jh_countries_live = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [13]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,2635417,127417,720631
Brazil,1402041,59594,788318
Russia,646929,9306,411973


In [14]:
jh_countries_live['Active'] = jh_countries_live['Confirmed'] - jh_countries_live['Deaths'] - jh_countries_live['Recovered']

In [15]:
jh_countries_live['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_countries_live['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_countries_live['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [16]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,2635417,127417,720631,1787369,2020-07-01 04:33:52,38.289352,-91.609228
Brazil,1402041,59594,788318,554129,2020-07-01 04:33:52,-12.669522,-48.480493
Russia,646929,9306,411973,225650,2020-07-01 04:33:52,54.546312,62.120860
India,585493,17400,347979,220114,2020-07-01 04:33:52,23.356511,82.081064
United Kingdom,314160,43815,1370,268975,2020-07-01 04:33:52,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [17]:
jh_states_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [18]:
jh_states_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,393454,32032,0
California,231232,6082,0
New Jersey,171667,15035,0


In [19]:
jh_states_live['Active'] = jh_states_live['Confirmed'] - jh_states_live['Deaths'] - jh_states_live['Recovered']

In [20]:
jh_states_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_states_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_states_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [21]:
jh_states_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,393454,32032,0,361422,2020-07-01 04:33:52,42.671593,-75.579694
California,231232,6082,0,225150,2020-07-01 04:33:52,37.778248,-120.728666
New Jersey,171667,15035,0,156632,2020-07-01 04:33:52,40.286441,-74.622450
Texas,163060,2455,0,160605,2020-07-01 04:33:52,31.657778,-98.589862
Florida,152434,3505,0,148929,2020-07-01 04:33:52,28.940755,-82.700744


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [22]:
jh_counties_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,215179,23096,0
California,Los Angeles,103850,3371,0
Illinois,Cook,90536,4565,0
Arizona,Maricopa,48592,771,0
New York,Nassau,41807,2182,0


In [24]:
jh_counties_live['Active'] = jh_counties_live['Confirmed'] - jh_counties_live['Deaths'] - jh_counties_live['Recovered']


In [25]:
jh_counties_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_counties_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_counties_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_counties_live['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [26]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,215179,23096,0,192083,2020-07-01 04:33:52,40.767273,-73.971526,36061.0
California,Los Angeles,103850,3371,0,100479,2020-07-01 04:33:52,34.308284,-118.228241,6037.0
Illinois,Cook,90536,4565,0,85971,2020-07-01 04:33:52,41.841448,-87.816588,17031.0
Arizona,Maricopa,48592,771,0,47821,2020-07-01 04:33:52,33.348359,-112.491815,4013.0
New York,Nassau,41807,2182,0,39625,2020-07-01 04:33:52,40.740665,-73.589419,36059.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [27]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_cases_global_hist.iloc[:, 4:].columns

In [28]:
jh_cases_countries_hist_ts = pd.pivot_table(jh_cases_global_hist, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.transpose()
jh_cases_countries_hist_ts.index = pd.to_datetime(jh_cases_countries_hist_ts.index)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.sort_index()

In [29]:
jh_cases_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,30967,2402,13273,855,267,69,59933,24645,7764,17654,...,312640,929,7948,5297,355,1990,10,1118,1557,567
2020-06-29,31238,2466,13571,855,276,69,62268,25127,7834,17723,...,313470,932,8222,5530,355,2185,10,1128,1568,574
2020-06-30,31517,2535,13907,855,284,69,64530,25542,7920,17766,...,314160,936,8503,5832,355,2428,10,1158,1594,591


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [30]:
jh_deaths_countries_hist_ts = pd.pivot_table(jh_deaths_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.transpose()
jh_deaths_countries_hist_ts.index = pd.to_datetime(jh_deaths_countries_hist_ts.index)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.sort_index()

In [31]:
jh_deaths_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,721,55,897,52,11,3,1232,426,104,702,...,43634,27,22,44,0,4,1,302,22,6
2020-06-29,733,58,905,52,11,3,1280,433,104,703,...,43659,27,23,48,0,5,1,304,22,7
2020-06-30,746,62,912,52,13,3,1307,443,104,705,...,43815,27,26,51,0,8,1,312,24,7


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [32]:
jh_recovered_countries_hist_ts = pd.pivot_table(jh_recovered_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.transpose()
jh_recovered_countries_hist_ts.index = pd.to_datetime(jh_recovered_countries_hist_ts.index)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.sort_index()

In [33]:
jh_recovered_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,12604,1384,9371,799,81,22,20134,13116,7007,16401,...,1364,818,5329,1649,330,447,8,430,1311,142
2020-06-29,13934,1438,9674,799,93,22,21138,13297,7037,16420,...,1368,822,5496,1649,335,447,8,432,1311,152
2020-06-30,14131,1459,9897,799,93,22,22028,14048,7040,16478,...,1370,824,5682,1649,335,451,8,488,1329,162


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [34]:
jh_cases_counties_hist.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,1075,1097,1140,1182,1265,1326,1389,1408,1453,1495
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,168,168,171,179,183,184,188,189,197,200
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,237,237,243,248,255,260,264,266,273,277
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,147,148,147,150,153,159,160,161,165,166


In [36]:
date_columns = jh_cases_counties_hist.iloc[:,11:].columns

In [37]:
jh_cases_states_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.transpose()
jh_cases_states_hist_ts.index = pd.to_datetime(jh_cases_states_hist_ts.index)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.sort_index()

In [38]:
jh_cases_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-28,35441,880,0,73920,19818,215296,32290,46303,11226,49,...,40172,150152,21100,1202,81,61736,31752,2832,27743,1417
2020-06-29,37175,901,0,74545,20257,223646,32494,46362,11376,49,...,42297,156706,21664,1208,81,62189,32253,2870,28058,1450
2020-06-30,38045,937,0,79228,20777,231232,32698,46514,11474,49,...,43509,163060,22217,1208,81,62787,32824,2905,28659,1487


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [39]:
jh_cases_counties_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.transpose()
jh_cases_counties_hist_ts.index = pd.to_datetime(jh_cases_counties_hist_ts.index)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.sort_index()

In [40]:
jh_cases_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-28         497     559     314  159    186     353    599     237   
2020-06-29         521     626     319  162    196     360    602     261   
2020-06-30         530     663     322  167    204     360    604     268   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-28          547       67  ...      37      2       26        6   
2020-06-29          571       70  ...      40      2       26        6   
2020-06-30          576       71  ...      50      3       27        6   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-28             82   123   168          0       39      1  
2020-06-29             86   128   174          0       39      2  
2020-06-30             90   129   176          0       39      2  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [41]:
jh_deaths_states_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.transpose()
jh_deaths_states_hist_ts.index = pd.to_datetime(jh_deaths_states_hist_ts.index)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.sort_index()

In [42]:
jh_deaths_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-28,919,14,0,1594,264,5932,1676,4316,507,0,...,584,2402,167,56,6,1732,1310,93,777,20
2020-06-29,929,14,0,1598,265,5983,1681,4320,507,0,...,592,2416,168,56,6,1740,1320,93,777,20
2020-06-30,950,14,0,1645,270,6082,1690,4322,509,0,...,604,2455,172,56,6,1763,1332,93,784,20


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [43]:
jh_deaths_counties_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.transpose()
jh_deaths_counties_hist_ts.index = pd.to_datetime(jh_deaths_counties_hist_ts.index)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.sort_index()

In [44]:
jh_deaths_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-28          11       9       1    1      1      10     27       5   
2020-06-29          11       9       1    1      1       9     27       5   
2020-06-30          11       9       1    1      1       9     27       5   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-28           27        7  ...       0      0        0        0   
2020-06-29           27        7  ...       0      0        0        0   
2020-06-30           27        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-28              0     0     0         19        0      0  
2020-06-29              0     0     0         19        0      0  
2020-06-30              0     0     0         19        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [45]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_countries_live.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [46]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_states_live.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [47]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_counties_live.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [48]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_cases_countries_hist_ts.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_deaths_countries_hist_ts.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_recovered_countries_hist_ts.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_cases_states_hist_ts.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_cases_counties_hist_ts.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_deaths_states_hist_ts.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_deaths_counties_hist_ts.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [55]:
jh_cases_countries_hist_pct = jh_cases_countries_hist_ts.pct_change()

In [56]:
jh_cases_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,0.011465,0.030901,0.023519,0.0,0.030888,0.061538,0.037909,0.030783,0.010148,0.004209,...,0.002929,0.005411,0.034626,0.032554,0.0,0.096419,0.0,0.013599,0.016982,0.000000
2020-06-29,0.008751,0.026644,0.022452,0.0,0.033708,0.000000,0.038960,0.019558,0.009016,0.003908,...,0.002655,0.003229,0.034474,0.043987,0.0,0.097990,0.0,0.008945,0.007065,0.012346
2020-06-30,0.008931,0.027981,0.024759,0.0,0.028986,0.000000,0.036327,0.016516,0.010978,0.002426,...,0.002201,0.004292,0.034177,0.054611,0.0,0.111213,0.0,0.026596,0.016582,0.029617


In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_cases_countries_hist_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [58]:
jh_deaths_countries_hist_pct = jh_deaths_countries_hist_ts.pct_change()

In [59]:
jh_deaths_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,0.025605,0.037736,0.005605,0.0,0.100000,0.0,0.020713,0.011876,0.0,0.002857,...,0.000826,0.038462,0.100000,0.047619,NaN,0.00,0.0,0.020270,0.047619,0.000000
2020-06-29,0.016644,0.054545,0.008919,0.0,0.000000,0.0,0.038961,0.016432,0.0,0.001425,...,0.000573,0.000000,0.045455,0.090909,NaN,0.25,0.0,0.006623,0.000000,0.166667
2020-06-30,0.017735,0.068966,0.007735,0.0,0.181818,0.0,0.021094,0.023095,0.0,0.002845,...,0.003573,0.000000,0.130435,0.062500,NaN,0.60,0.0,0.026316,0.090909,0.000000


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_deaths_countries_hist_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [61]:
jh_recovered_countries_hist_pct = jh_recovered_countries_hist_ts.pct_change()

In [62]:
jh_recovered_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,0.180813,0.028232,0.018366,0.0,0.000000,0.0,0.051768,0.015878,0.002002,0.001833,...,0.000000,0.000000,0.016985,0.242653,0.000000,0.002242,0.0,0.031175,0.06326,0.000000
2020-06-29,0.105522,0.039017,0.032334,0.0,0.148148,0.0,0.049866,0.013800,0.004281,0.001158,...,0.002933,0.004890,0.031338,0.000000,0.015152,0.000000,0.0,0.004651,0.00000,0.070423
2020-06-30,0.014138,0.014604,0.023051,0.0,0.000000,0.0,0.042104,0.056479,0.000426,0.003532,...,0.001462,0.002433,0.033843,0.000000,0.000000,0.008949,0.0,0.129630,0.01373,0.065789


In [63]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_recovered_countries_hist_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [64]:
jh_cases_states_hist_pct = jh_cases_states_hist_ts.pct_change()

In [65]:
jh_cases_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-28,0.010204,0.034078,NaN,0.054133,0.026308,0.022376,0.008873,0.002099,0.012172,0.0,...,0.000000,0.029186,0.022882,0.001667,0.0,0.007984,0.011081,0.017973,0.016749,0.017960
2020-06-29,0.048926,0.023864,NaN,0.008455,0.022152,0.038784,0.006318,0.001274,0.013362,0.0,...,0.052898,0.043649,0.026730,0.004992,0.0,0.007338,0.015779,0.013418,0.011354,0.023289
2020-06-30,0.023403,0.039956,NaN,0.062821,0.025670,0.033920,0.006278,0.003279,0.008615,0.0,...,0.028655,0.040547,0.025526,0.000000,0.0,0.009616,0.017704,0.012195,0.021420,0.025517


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_cases_states_hist_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_cases_counties_hist_pct = jh_cases_counties_hist_ts.pct_change()

In [68]:
jh_cases_counties_hist_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-28      0.010163  0.037106  0.000000  0.006329  0.005405 -0.002825   
2020-06-29      0.048290  0.119857  0.015924  0.018868  0.053763  0.019830   
2020-06-30      0.017274  0.059105  0.009404  0.030864  0.040816  0.000000   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-06-28      0.003350  0.000000  0.000000  0.015152  ...  0.193548    0.0   
2020-06-29      0.005008  0.101266  0.043876  0.044776  ...  0.081081    0.0   
2020-06-30      0.003322  0.026820  0.008757  0.014286  ...  0.250000    0.5   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-06-28      0.083333      0.0   0.012346  0.033613  0.005988        NaN   
2020-06-29      0.000000      0.0   0.048780  0.040650  0.035714        NaN   
2020-06-30      0.038462      0.0   0.046512  0.007812  0.011494        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-06-28          0.0    0.0  
2020-06-29          0.0    1.0  
2020-06-30          0.0    0.0  

[3 rows x 3254 columns]

In [69]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_cases_counties_hist_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [70]:
jh_deaths_states_hist_pct = jh_deaths_states_hist_ts.pct_change()

In [71]:
jh_deaths_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-28,0.000000,0.0,NaN,0.008861,0.019305,0.005594,0.001195,0.001160,0.000000,NaN,...,0.000000,0.005863,0.000000,0.0,0.0,0.004640,0.000000,0.0,0.000000,0.0
2020-06-29,0.010881,0.0,NaN,0.002509,0.003788,0.008597,0.002983,0.000927,0.000000,NaN,...,0.013699,0.005828,0.005988,0.0,0.0,0.004619,0.007634,0.0,0.000000,0.0
2020-06-30,0.022605,0.0,NaN,0.029412,0.018868,0.016547,0.005354,0.000463,0.003945,NaN,...,0.020270,0.016142,0.023810,0.0,0.0,0.013218,0.009091,0.0,0.009009,0.0


In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_deaths_states_hist_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [73]:
jh_deaths_counties_hist_pct = jh_deaths_counties_hist_ts.pct_change()

In [74]:
jh_deaths_counties_hist_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-28         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-06-29         0.0     0.0     0.0  0.0    0.0    -0.1    0.0     0.0   
2020-06-30         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-28          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-06-29          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-06-30          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-28            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-29            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-30            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [75]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_deaths_counties_hist_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [76]:
jh_cases_countries_hist_pct_ewma = jh_cases_countries_hist_pct.ewm(span=3).mean()

In [77]:
jh_cases_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,0.009920,0.030698,0.021591,7.247532e-07,0.077621,0.124519,0.043211,0.030671,0.008530,0.003485,...,0.003136,0.006868,0.034354,0.045818,0.002071,0.117198,0.000868,0.016750,0.012450,0.008683
2020-06-29,0.009335,0.028671,0.022021,3.623766e-07,0.055665,0.062260,0.041086,0.025114,0.008773,0.003697,...,0.002895,0.005049,0.034414,0.044903,0.001036,0.107594,0.000434,0.012847,0.009757,0.010514
2020-06-30,0.009133,0.028326,0.023390,1.811883e-07,0.042325,0.031130,0.038706,0.020815,0.009875,0.003061,...,0.002548,0.004670,0.034295,0.049757,0.000518,0.109403,0.000217,0.019721,0.013170,0.020066


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_cases_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [79]:
jh_deaths_countries_hist_pct_ewma = jh_deaths_countries_hist_pct.ewm(span=3).mean()

In [80]:
jh_deaths_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,0.027153,0.038413,0.007041,2.393536e-06,0.051227,2.667657e-23,0.022853,0.020791,0.000457,0.002679,...,0.001969,0.020567,0.053290,0.041397,NaN,0.083333,0.0,0.020020,0.054589,0.001953
2020-06-29,0.021898,0.046479,0.007980,1.196768e-06,0.025613,1.333829e-23,0.030907,0.018611,0.000228,0.002052,...,0.001271,0.010284,0.049372,0.066153,NaN,0.166667,0.0,0.013321,0.027295,0.084310
2020-06-30,0.019817,0.057722,0.007857,5.983839e-07,0.103716,6.669144e-24,0.026001,0.020853,0.000114,0.002448,...,0.002422,0.005142,0.089903,0.064327,NaN,0.383333,0.0,0.019819,0.059102,0.042155


In [81]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_deaths_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [82]:
jh_recovered_countries_hist_pct_ewma = jh_recovered_countries_hist_pct.ewm(span=3).mean()

In [83]:
jh_recovered_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-28,0.106262,0.031172,0.016757,0.000375,0.003939,0.000012,0.072488,0.039934,0.002541,0.001722,...,0.001586,0.000245,0.026701,0.125951,0.000461,0.001810,0.000005,0.034711,0.033205,0.051234
2020-06-29,0.105892,0.035095,0.024546,0.000187,0.076044,0.000006,0.061177,0.026867,0.003411,0.001440,...,0.002259,0.002567,0.029019,0.062976,0.007806,0.000905,0.000003,0.019681,0.016603,0.060828
2020-06-30,0.060015,0.024849,0.023798,0.000094,0.038022,0.000003,0.051641,0.041673,0.001919,0.002486,...,0.001861,0.002500,0.031431,0.031488,0.003903,0.004927,0.000001,0.074655,0.015166,0.063309


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_recovered_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [85]:
jh_cases_states_hist_pct_ewma = jh_cases_states_hist_pct.ewm(span=3).mean()

In [86]:
jh_cases_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-28,0.019156,0.028866,NaN,0.052750,0.030319,0.022661,0.008579,0.002203,0.008964,1.678427e-32,...,0.011976,0.035759,0.026572,0.002989,0.005016,0.009099,0.014019,0.017795,0.017893,0.020805
2020-06-29,0.034041,0.026365,NaN,0.030603,0.026235,0.030723,0.007448,0.001738,0.011163,8.392137e-33,...,0.032437,0.039704,0.026651,0.003990,0.002508,0.008218,0.014899,0.015606,0.014623,0.022047
2020-06-30,0.028722,0.033160,NaN,0.046712,0.025953,0.032321,0.006863,0.002508,0.009889,4.196069e-33,...,0.030546,0.040126,0.026089,0.001995,0.001254,0.008917,0.016301,0.013901,0.018022,0.023782


In [87]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_cases_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [88]:
jh_cases_counties_hist_pct_ewma = jh_cases_counties_hist_pct.ewm(span=3).mean()

In [89]:
jh_cases_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-28      0.015846  0.051941  0.011306  0.025937  0.020015  0.007208   
2020-06-29      0.032068  0.085899  0.013615  0.022402  0.036889  0.013519   
2020-06-30      0.024671  0.072502  0.011510  0.026633  0.038853  0.006760   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-06-28      0.005827  0.016179  0.004902  0.024245  ...  0.200630   
2020-06-29      0.005418  0.058722  0.024389  0.034510  ...  0.140855   
2020-06-30      0.004370  0.042771  0.016573  0.024398  ...  0.195428   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-06-28      0.007813  0.062899  0.052116   0.034228  0.021039  0.012080   
2020-06-29      0.003906  0.031449  0.026058   0.041504  0.030845  0.023897   
2020-06-30      0.251953  0.034955  0.013029   0.044008  0.019329  0.017696   

Province_State                                  
Admin2         Unassigned      Washakie Weston  
2020-06-28            NaN  1.742783e-09   0.00  
2020-06-29            NaN  8.713915e-10   0.50  
2020-06-30            NaN  4.356958e-10   0.25  

[3 rows x 3254 columns]

In [90]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_cases_counties_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [91]:
jh_deaths_states_hist_pct_ewma = jh_deaths_states_hist_pct.ewm(span=3).mean()

In [92]:
jh_deaths_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-28,0.006689,0.020834,NaN,0.018430,0.025715,0.007194,0.001324,0.001383,0.002814,NaN,...,0.007759,0.010638,0.004024,0.000009,1.154650e-15,0.008861,0.002274,0.003289,0.004802,0.000109
2020-06-29,0.008785,0.010417,NaN,0.010470,0.014752,0.007896,0.002154,0.001155,0.001407,NaN,...,0.010729,0.008233,0.005006,0.000004,5.773251e-16,0.006740,0.004954,0.001644,0.002401,0.000054
2020-06-30,0.015695,0.005209,NaN,0.019941,0.016810,0.012221,0.003754,0.000809,0.002676,NaN,...,0.015499,0.012188,0.014408,0.000002,2.886626e-16,0.009979,0.007022,0.000822,0.005705,0.000027


In [93]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_deaths_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [94]:
jh_deaths_counties_hist_pct_ewma = jh_deaths_counties_hist_pct.ewm(span=3).mean()

In [95]:
jh_deaths_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga       Baldwin Barbour Bibb Blount   Bullock   Butler   
2020-06-28      0.033173  4.577324e-13     0.0  0.0    0.0  0.003594  0.00064   
2020-06-29      0.016587  2.288662e-13     0.0  0.0    0.0 -0.048203  0.00032   
2020-06-30      0.008293  1.144331e-13     0.0  0.0    0.0 -0.024101  0.00016   

Province_State                                ... Wyoming                  \
Admin2           Calhoun  Chambers  Cherokee  ...    Park Platte Sheridan   
2020-06-28      0.003947  0.000005  0.002801  ...     NaN    NaN      NaN   
2020-06-29      0.001973  0.000003  0.001401  ...     NaN    NaN      NaN   
2020-06-30      0.000987  0.000001  0.000700  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-28          NaN        NaN   NaN   NaN   0.000115      NaN    NaN  
2020-06-29          NaN        NaN   NaN   NaN   0.000058      NaN    NaN  
2020-06-30          NaN        NaN   NaN   NaN   0.000029      NaN    NaN  

[3 rows x 3254 columns]

In [96]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_deaths_counties_hist_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

# TODO - CONTINUE FROM HERE

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/states | [CSV](https://covidtracking.com/api/states.csv)  
States daily 4 pm ET - /api/states/daily | [CSV](http://covidtracking.com/api/states/daily.csv)  
States info - /api/states/info | [CSV](https://covidtracking.com/api/states/info.csv)  
US current - /api/us | [CSV](https://covidtracking.com/api/us.csv)  
US daily - /api/us/daily | [CSV](https://covidtracking.com/api/us/daily.csv)  
Counties- /api/counties | [CSV](https://covidtracking.com/api/counties.csv)  